In [ ]:
# Uncomment and run the below lines in terminal to install the yfinance APIs we'll use
# Do this only once. Type "pip show yfinance" into terminal to check if it's already been installed. 
# The installation command can also be run in terminal

# pip install yfinance
# pip install yfinance yahoo_fin

In [30]:
# Importing libraries to use
import yfinance as yf
import pandas as pd
import os
from datetime import datetime, timedelta
from pytz import timezone
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from yahoo_fin.stock_info import tickers_nasdaq as get_nasdaq_tickers

In [37]:
# This method pulls an exhaustive list of stocks within a certain market cap
def get_tickers_by_market_cap(min_cap, max_cap):
    # Fetch all NASDAQ tickers. This should cover a significant portion of US-listed stocks.
    all_tickers = get_nasdaq_tickers()
    selected_tickers = []

    for ticker in all_tickers:
        try:
            stock = yf.Ticker(ticker)
            market_cap = stock.info["marketCap"]
            
            if min_cap <= market_cap <= max_cap:
                selected_tickers.append(ticker)

        except Exception as e:
            print(f"Could not fetch data for {ticker} due to {e}")

    return selected_tickers

In [ ]:
%%time

min_market_cap = 100e6  # e.g., 100 million
max_market_cap = 50e9  # e.g., 50 billion

tickers = get_tickers_by_market_cap(min_market_cap, max_market_cap)

# Write tickers to a txt file
with open("filtered_tickers.txt", "w") as f:
    for ticker in tickers:
        f.write(f"{ticker}\n")

print(f"{len(tickers)} tickers written to filtered_tickers.txt")

Could not fetch data for AACIU due to 'marketCap'
Could not fetch data for AACIW due to 'marketCap'
Could not fetch data for AADR due to 'marketCap'
Could not fetch data for AAPB due to 'marketCap'
Could not fetch data for AAPD due to 'marketCap'
Could not fetch data for AAPU due to 'marketCap'
Could not fetch data for AAXJ due to 'marketCap'
Could not fetch data for ABLLW due to 'marketCap'
Could not fetch data for ABLVW due to 'marketCap'
Could not fetch data for ACABU due to 'marketCap'
Could not fetch data for ACABW due to 'marketCap'
Could not fetch data for ACACU due to 'marketCap'
Could not fetch data for ACACW due to 'marketCap'
Could not fetch data for ACAHU due to 'marketCap'
Could not fetch data for ACAHW due to 'marketCap'
Could not fetch data for ACAXR due to 'marketCap'
Could not fetch data for ACAXU due to 'marketCap'
Could not fetch data for ACAXW due to 'marketCap'
Could not fetch data for ACBAU due to 'marketCap'
Could not fetch data for ACBAW due to 'marketCap'
Could

In [ ]:
# Detect potential trading halts in minute data.
# Returns a list of potential halt times.
def detect_halt(data_minute):
    data_minute['Time Gap'] = data_minute.index.to_series().diff().fillna(pd.Timedelta(seconds=0))
    potential_halts = data_minute[data_minute['Time Gap'] > pd.Timedelta(minutes=1)]
    return potential_halts.index.tolist()

In [ ]:
# Method to check if the ticker fulfills the halt condition:
# daily volume of 100,000 or higher that halt down. Then, when they re-open, 
# they conclude lower than the price of the stock when it was halted.
def check_halt_condition(stock_symbol, halt_price, resume_time, conclude_time, ticker_data):
    data = ticker_data[stock_symbol]

    # Check if resume_time and conclude_time are in data
    if resume_time not in data.index or conclude_time not in data.index:
        return False

    resume_price = data.at[resume_time, 'Open']
    conclude_price = data.at[conclude_time, 'Close']

    if halt_price > resume_price and conclude_price < halt_price:
        return True
    return False

In [ ]:
datetime.now()

In [ ]:
# Checks and pulls data for all the relevant tickers
# Uses current date if you don't pass a specific date
def find_stocks(tickers, date=None):
    candidates = []
    ticker_data = {}

    eastern = timezone('US/Eastern')  # Assuming data is in Eastern Time, adjust as needed

    # If no date is passed, use today's date
    if date is None:
        date = datetime.now().date()

    for ticker in tickers:
        try:
            data_daily = yf.download(ticker, start=date, end=date + timedelta(days=1))
            if data_daily['Volume'][-1] >= 100000:
                data_minute = yf.download(ticker, interval="1m", start=date, end=date + timedelta(days=1))
                data_minute = data_minute.tz_convert(eastern)  # Convert to Eastern Time

                # Handle empty data
                if data_minute.empty:
                    print(f"No minute data for {ticker} on {date}.")
                    continue
                    
                ticker_data[ticker] = data_minute
                
                # Detect potential halts
                halt_times = detect_halt(data_minute)
                
                for halt_time in halt_times:
                    # For demonstration purposes, assume that trading resumes in the next available minute after the halt.
                    resume_time = data_minute.loc[halt_time:].index[1]
                    conclude_time = resume_time + pd.Timedelta(minutes=1)

                    halt_price = data_minute.at[halt_time, 'Close']

                    if check_halt_condition(ticker, halt_price, resume_time, conclude_time, ticker_data):
                        candidates.append(ticker)

                

        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    return candidates, ticker_data

In [ ]:
# Reads tickers entered line by line into a txt file
def read_tickers(filename):
    with open(filename,'r') as file:
        lines = [line.rstrip() for line in file]
    return lines

In [ ]:
# This is the cell that makes the magic happen
# tickers = ['NBY', 'RETO','SPGC']  # Add tickers you are interested in
tickers=read_tickers('/Users/georgepandya/Desktop/Stock Strategies/StockStuff/tickers.txt') #Or Read from a file

date_to_check = datetime(2023, 9, 11).date()
result, all_data = find_stocks(tickers,date_to_check)

print(f"Stocks to short: {result}")

# Plot each of the relevant stocks into 
plt.figure(figsize=(10,6))
fig, ax = plt.subplots()

for ticker, df in all_data.items():
    # plt.plot(#df['Datetime'].strftime("%H:%M:%S"),
    #          df['Open'],label=ticker)
    df['Open'].plot(ax=ax,label=ticker)
    print(ticker+'Meets halt short condition. Data outputted to CSV')
    df.to_csv(ticker+'.csv')
# Format the plot
# fig, ax = plt.subplots()
# ax.plot(dates, values)

plt.title(f'Trading data for {datetime.now().date().strftime("%m/%d/%Y")}')
plt.xlabel('Time')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S %Z'))
plt.xticks(rotation=45)
plt.ylabel('Stock Price')
plt.legend()
plt.tight_layout()
plt.show()